In [2]:
from osgeo import gdal
import os


In [9]:
import os
from osgeo import gdal

def cut_image_into_blocks(image_path, output_folder):
    dataset = gdal.Open(image_path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    data_type = dataset.GetRasterBand(1).DataType

    block_width = width // 4  # Dividir en 4 columnas
    block_height = height // 4  # Dividir en 4 filas

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for row in range(4):
        for col in range(4):
            x_offset = col * block_width
            y_offset = row * block_height

            output_name = f"{os.path.splitext(os.path.basename(image_path))[0][:17]}_block_{row}_{col}.tif"
            output_path = os.path.join(output_folder, output_name)

            # Construye el comando gdal_translate
            cmd = f"gdal_translate -srcwin {x_offset} {y_offset} {block_width} {block_height} -of GTiff -co PHOTOMETRIC=RGB -ot UInt16 {image_path} {output_path}"

            # Ejecuta el comando en la línea de comandos
            os.system(cmd)


In [11]:
nombres_de_puertos = ["TPCallao"]

for puerto in nombres_de_puertos:
    input_folder = f"/home/farix/python/dataset/{puerto}/ALTARESO"
    output_folder = f"/home/farix/python/dataset/{puerto}/BLOQUES"

    # Lista los archivos tif en la carpeta de entrada
    tif_files = [file for file in os.listdir(input_folder) if file.endswith(".tif")]

    for tif_file in tif_files:
        input_path = os.path.join(input_folder, tif_file)

        # Extraer el nombre del archivo sin extensión
        file_name_without_extension = os.path.splitext(tif_file)[0]

        # Crear el nombre de la carpeta correspondiente
        output_folder_for_file = os.path.join(output_folder, file_name_without_extension)

        # Verificar si la carpeta ya existe
        if not os.path.exists(output_folder_for_file):
            os.makedirs(output_folder_for_file)

            # Realizar el corte en bloques si la carpeta no existe
            cut_image_into_blocks(input_path, output_folder_for_file)
        else:
            print(f"La carpeta {output_folder_for_file} ya existe, omitiendo el corte para el archivo {tif_file}.")

Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 30776, 32636
0...10...20...30...40...50...60...70..

In [1]:
import os
import errno
import rasterio

def convertir_tif_a_png(folder_path_tif):
    # Cambia la ruta para guardar las imágenes png
    folder_path_png = folder_path_tif.replace('BLOQUES', 'BLOQUESPNG')  # Añadir "/" al final

    # Crea folder para guardar las imágenes png
    try:
        os.mkdir(folder_path_png)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    # Recorre cada archivo en la carpeta de imágenes TIF
    for entry in os.listdir(folder_path_tif):
        if entry.lower().endswith('.tif') or entry.lower().endswith('.tiff'):
            print(entry)
            # Para GDAL, toma los valores máximos y mínimos de cada banda en la imagen TIF
            # Con rasterio
            RGB = os.path.join(folder_path_tif, entry)
            imagen_tif = rasterio.open(RGB)

            # Lectura del valor mínimo de la banda 1
            arrayMin1 = imagen_tif.read(1)
            banda1Min = arrayMin1.min()
            # Lectura del valor máximo de la banda 1
            arrayMax1 = imagen_tif.read(1)
            banda1Max = arrayMax1.max()

            # Lectura del valor mínimo de la banda 2
            arrayMin2 = imagen_tif.read(2)
            banda2Min = arrayMin2.min()

            # Lectura del valor máximo de la banda 2
            arrayMax2 = imagen_tif.read(2)
            banda2Max = arrayMax2.max()

            # Lectura del valor mínimo de la banda 3
            arrayMin3 = imagen_tif.read(3)
            banda3Min = arrayMin3.min()

            # Lectura del valor máximo de la banda 3
            arrayMax3 = imagen_tif.read(3)
            banda3Max = arrayMax3.max()
            
            # Cierra el archivo TIF
            imagen_tif.close()
            
            # Conversión a PNG
            without_ext = os.path.splitext(entry)[0]
            file_PNG = os.path.join(folder_path_png, without_ext + '.png')
            print(file_PNG)

            to_png = ('gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 ' + str(banda1Min) + ' ' + str(
                banda1Max) + ' -scale_2 ' + str(banda2Min) + ' ' + str(banda2Max) + ' -scale_3 ' + str(
                banda3Min) + ' ' + str(banda3Max) + ' -a_nodata none ' + RGB + ' ' + file_PNG)

            print(to_png)

            # Ejecuta en un bash de Linux para la conversión
            os.system(to_png)

    # Elimina los archivos XML generados durante la conversión
    delete_xml = "rm " + folder_path_png + "*.aux.xml"
    print(delete_xml)
    os.system(delete_xml)

def convertir_estructura_de_carpetas(carpeta_raiz_tif):
    # Itera sobre las subcarpetas en la carpeta raíz
    for subdir, _, _ in os.walk(carpeta_raiz_tif):
        # Llama a la función de conversión para cada subcarpeta
        convertir_tif_a_png(subdir)

# Carpeta raíz que contiene subcarpetas, cada una representando una imagen
carpeta_raiz_tif = '/home/farix/python/dataset/TPCallao/BLOQUES/'

# Llama a la función para convertir toda la estructura de carpetas
convertir_estructura_de_carpetas(carpeta_raiz_tif)


rm /home/farix/python/dataset/TPCallao/BLOQUESPNG/*.aux.xml
IMG_PER1_20200429_block_0_0.tif
/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_0.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 0 2856 -scale_2 0 4760 -scale_3 0 7227 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_0.tif /home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_0.png
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
IMG_PER1_20200429_block_0_3.tif
/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_3.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 0 0 -scale_2 0 0 -scale_3 0 0 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429

rm: cannot remove '/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpened*.aux.xml': No such file or directory


/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20230129153659_ORT_P_000054_pansharpened/IMG_PER1_20230129_block_0_1.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 0 3607 -scale_2 0 3784 -scale_3 0 4401 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20230129153659_ORT_P_000054_pansharpened/IMG_PER1_20230129_block_0_1.tif /home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20230129153659_ORT_P_000054_pansharpened/IMG_PER1_20230129_block_0_1.png
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
IMG_PER1_20230129_block_1_1.tif
/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20230129153659_ORT_P_000054_pansharpened/IMG_PER1_20230129_block_1_1.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 1 4502 -scale_2 1 4752 -scale_3 1 5458 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20230129153659_ORT_P_000054_pansharpened/IMG_PER1_20230129_block_1_1.tif /home/farix/python

rm: cannot remove '/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20230129153659_ORT_P_000054_pansharpened*.aux.xml': No such file or directory


In [1]:
import os
import errno
import rasterio
from pathlib import Path

def convertir_tif_a_png(folder_path_tif):
    # Cambia la ruta para guardar las imágenes png
    folder_path_png = folder_path_tif.replace('BLOQUES', 'BLOQUESPNG')  # Añadir "/" al final

    # Crea folder para guardar las imágenes png
    try:
        os.mkdir(folder_path_png)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    # Recorre cada archivo en la carpeta de imágenes TIF
    for entry in os.listdir(folder_path_tif):
        if os.path.isfile(os.path.join(folder_path_tif, entry)):
            print(entry)
            # Para GDAL, toma los valores máximos y mínimos de cada banda en la imagen TIF
            # Con rasterio
            RGB = os.path.join(folder_path_tif, entry)
            imagen_tif = rasterio.open(RGB)

            # Lectura del valor mínimo de la banda 1
            arrayMin1 = imagen_tif.read(1)
            banda1Min = arrayMin1.min()
            # Lectura del valor máximo de la banda 1
            arrayMax1 = imagen_tif.read(1)
            banda1Max = arrayMax1.max()

            # Lectura del valor mínimo de la banda 2
            arrayMin2 = imagen_tif.read(2)
            banda2Min = arrayMin2.min()

            # Lectura del valor máximo de la banda 2
            arrayMax2 = imagen_tif.read(2)
            banda2Max = arrayMax2.max()

            # Lectura del valor mínimo de la banda 3
            arrayMin3 = imagen_tif.read(3)
            banda3Min = arrayMin3.min()

            # Lectura del valor máximo de la banda 3
            arrayMax3 = imagen_tif.read(3)
            banda3Max = arrayMax3.max()
            # Cierra el archivo TIF
            imagen_tif.close()
            # Conversión a PNG

            # La carpeta de salida de los archivos png
            # Archivo de salida, el nombre es el mismo que el archivo de entrada con extensión PNG
            without_ext = os.path.splitext(entry)[0]
            file_PNG = folder_path_png + without_ext + '.png'
            print(file_PNG)

            to_png = ('gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 ' + str(banda1Min) + ' ' + str(
                banda1Max) + ' -scale_2 ' + str(banda2Min) + ' ' + str(banda2Max) + ' -scale_3 ' + str(
                banda3Min) + ' ' + str(banda3Max) + ' -a_nodata none ' + RGB + ' ' + file_PNG)

            print(to_png)

            # Ejecuta en un bash de Linux para la conversión
            os.system(to_png)

    # Elimina los archivos XML generados durante la conversión
    delete_xml = "rm " + folder_path_png + "*.aux.xml"
    print(delete_xml)
    os.system(delete_xml)

def convertir_estructura_de_carpetas(carpeta_raiz_tif):
    # Itera sobre las subcarpetas en la carpeta raíz
    for subdir, _, _ in os.walk(carpeta_raiz_tif):
        # Verifica si la subcarpeta contiene archivos TIFF
        archivos_tif = [f for f in os.listdir(subdir) if f.lower().endswith('.tif') or f.lower().endswith('.tiff')]
        if archivos_tif:
            # Llama a la función de conversión para la subcarpeta
            convertir_tif_a_png(subdir)

# Carpeta raíz que contiene subcarpetas, cada una representando una imagen
carpeta_raiz_tif = '/home/farix/python/dataset/TPCallao/BLOQUES/'

# Llama a la función para convertir toda la estructura de carpetas
convertir_estructura_de_carpetas(carpeta_raiz_tif)


IMG_PER1_20200429_block_0_0.tif
/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpenedIMG_PER1_20200429_block_0_0.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 0 2856 -scale_2 0 4760 -scale_3 0 7227 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_0.tif /home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpenedIMG_PER1_20200429_block_0_0.png
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
IMG_PER1_20200429_block_0_3.tif
/home/farix/python/dataset/TPCallao/BLOQUESPNG/IMG_PER1_20200429151954_ORT_P_000535_pansharpenedIMG_PER1_20200429_block_0_3.png
gdal_translate -ot Byte -of PNG -b 1 -b 2 -b 3 -scale_1 0 0 -scale_2 0 0 -scale_3 0 0 -a_nodata none /home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_0_3.ti

RasterioIOError: '/home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_1_1.tif.aux.xml' not recognized as a supported file format.

In [5]:
# def cut_image_into_blocks(image_path, output_folder):
#     dataset = gdal.Open(image_path)
#     width = dataset.RasterXSize
#     height = dataset.RasterYSize
#     data_type = dataset.GetRasterBand(1).DataType

#     block_width = width // 4  # Dividir en 4 columnas
#     block_height = height // 4  # Dividir en 4 filas

#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     for row in range(4):
#         for col in range(4):
#             x_offset = col * block_width
#             y_offset = row * block_height
#             output_name = f"block_{row}_{col}.tif"
#             output_path = os.path.join(output_folder, output_name)

#             # Construye el comando gdal_translate
#             cmd = f"gdal_translate -srcwin {x_offset} {y_offset} {block_width} {block_height} -of GTiff -co PHOTOMETRIC=RGB -ot UInt16 {image_path} {output_path}"

#             # Ejecuta el comando en la línea de comandos
#             os.system(cmd)

In [6]:
# input_image_path = "../imagen/rgbImagen.tif"
# output_folder = "../output_blocksv3"
# cut_image_into_blocks(input_image_path, output_folder)


Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.

In [7]:
import os
import numpy as np
from osgeo import gdal

def get_min_max_values(image_path):
    dataset = gdal.Open(image_path)
    num_bands = dataset.RasterCount
    min_values = []
    max_values = []

    for band_num in range(1, num_bands + 1):
        band = dataset.GetRasterBand(band_num)
        band_data = band.ReadAsArray()
        min_val = np.min(band_data)
        max_val = np.max(band_data)
        min_values.append(min_val)
        max_values.append(max_val)

    return min_values, max_values

def cut_image_into_blocks512x512(image_path, output_folder):
    min_values, max_values = get_min_max_values(image_path)
    dataset = gdal.Open(image_path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    data_type = dataset.GetRasterBand(1).DataType

    block_width = 512
    block_height = 512

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for y_offset in range(0, height, block_height):
        for x_offset in range(0, width, block_width):
            x_size = min(block_width, width - x_offset)
            y_size = min(block_height, height - y_offset)
            
            output_name = f"block_{y_offset}_{x_offset}.tif"
            output_path = os.path.join(output_folder, output_name)
            
            cmd = f"gdal_translate -srcwin {x_offset} {y_offset} {x_size} {y_size} -of GTiff -co PHOTOMETRIC=RGB -ot UInt16 -scale_1 {min_values[0]} {max_values[0]} -scale_2 {min_values[1]} {max_values[1]} -scale_3 {min_values[2]} {max_values[2]} {image_path} {output_path}"
            # Ejecuta el comando en la línea de comandos
            os.system(cmd)

In [3]:
import os
from osgeo import gdal

def cut_image_into_blocks512x512(image_path, output_folder):
    dataset = gdal.Open(image_path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    data_type = dataset.GetRasterBand(1).DataType

    block_width = 512
    block_height = 512

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for y_offset in range(0, height, block_height):
        for x_offset in range(0, width, block_width):
            x_size = min(block_width, width - x_offset)
            y_size = min(block_height, height - y_offset)
            
            output_name = f"block_{y_offset}_{x_offset}.tif"
            output_path = os.path.join(output_folder, output_name)
            
            cmd = f"gdal_translate -srcwin {x_offset} {y_offset} {x_size} {y_size} -of GTiff -co PHOTOMETRIC=RGB -ot UInt16 {image_path} {output_path}"
            # Ejecuta el comando en la línea de comandos
            os.system(cmd)



In [8]:
input_image_path = "/home/farix/python/dataset/TPCallao/BLOQUES/IMG_PER1_20200429151954_ORT_P_000535_pansharpened/IMG_PER1_20200429_block_1_1.tif"
output_folder = "/home/farix/python/dataset/TPCallao/BLOQUESPNGV2"
cut_image_into_blocks512x512(input_image_path, output_folder)


Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7694, 8159
0...10...20...30...40...50...60...70...80...90...100 - done.